In [1]:
import cartopy
import json
import os
import requests
import shutil
import sys
import zipfile

In [2]:
def fetch(url,comment=None,projection="PlateCarree",resolution_key="low",target_dir=None):

    background_images_dir = os.path.join(cartopy.__path__[0], "data", "raster", "natural_earth")
    
    # tmpfile is the path+name of the downloaded Natural Earth zipfile.
    tmpfile = url.split('/')[-1]
    
    # Not tested.
    if target_dir is not None:
         tmpfile = target_dir+tmpfile # Assume trailing '/' in target_dir as needed.

    try:
        f = open(tmpfile)
        f.close()
    except FileNotFoundError:
        print("File %s not found. Fetching."%tmpfile)
        r = requests.get(url,stream=True)
        with open(tmpfile,"wb") as tmp:
            for chunk in r.iter_content(chunk_size=1024):
                if chunk:
                    tmp.write(chunk)
                
    with zipfile.ZipFile(tmpfile,'r') as tmp:
        names = tmp.namelist()
        
        member_name = [i for i in names if 'tif' in i] # Only expect one raster
        member_name = member_name[0]
        image_name = member_name.split('/')[-1] # Expecting <something>/<image name>
        # Should check for correctness and existence here
        tmp.extract(member_name,background_images_dir)
        try:
            shutil.move(background_images_dir+'/'+member_name
                        ,background_images_dir+'/'
                        )
        except:
            print('%s probably already installed. Skipping copy.'%image_name)
            pass
        # print('removing %s'%(background_images_dir+'/'+member_name.split('/')[0],))
        try:
            shutil.rmtree(background_images_dir+'/'+member_name.split('/')[0])
        except:
            # Sometimes the image is not stored in a folder.
            pass
    
    images_json_entry = {
        '__source__'      : url
        ,'__projection__' : projection
        ,resolution_key   : image_name
    }
    
    if comment is not None:
        images_json_entry['__comment__'] = comment
    
    return images_json_entry

# Ref. https://thomasguymer.co.uk/blog/2018/2018-01-15/

def quote(s):
    return "'"+s+"'"


In [3]:

background_images_dir = os.path.join(cartopy.__path__[0], "data", "raster", "natural_earth")
print('background_images_dir: \n  ',background_images_dir)
background_filename = background_images_dir+'/images.json'

try:
    shutil.copyfile(background_filename,background_filename+'.sav')
    print('backed up: \n  ',background_filename+'.sav')
except IOError as e:
    print("Unable to save backup. %s"%e)
except:
    print("Unexpected error: ",sys.exc_info())

with open(background_filename,'r') as bg_file:
    background_images = json.loads(bg_file.read())

# target_dir = None
target_dir = "./tmp/"
os.makedirs(target_dir,exist_ok=True)
print('zipfiles stored to %s'%target_dir)

background_images['hypso_50m_sr_w'] \
    = fetch("https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/50m/raster/HYP_50M_SR_W.zip"
            ,comment    = 'Cross-blended Hypsometric with Shaded Relief and Water'
            ,target_dir = target_dir
           )

background_images['ne1_50m_sr_w'] \
    = fetch("https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/50m/raster/NE1_50M_SR_W.zip"
            ,comment='Natural Earth 1 with Shaded Relief and Water'
            ,target_dir = target_dir
           )

background_images['hypso_10m_sr_w_ob_dr'] \
    = fetch("https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/raster/HYP_HR_SR_OB_DR.zip"
            ,comment='Cross-blended Hypsometric with relief, water, ocean bottom, and drainages.'
            ,target_dir = target_dir
           )

background_images['ne1_10m_sr_w_dr'] \
    = fetch("https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/raster/NE1_HR_LC_SR_W_DR.zip"
            ,comment='Natural Earth 1 with Shaded Relief, Water, and Drainages'
            ,target_dir = target_dir
           )

# print('\nbackground_images:     ',background_images)
with open(background_filename,'w') as bg_file:
    bg_file.writelines(json.dumps(background_images,indent = 4, sort_keys=True))
    
print('\nBackground images loaded:')
for i in background_images:
    try:
        print("  %-35s [resolution='low']:  %s"%(quote(i),background_images[i]['low']))
    except:
        pass
    try:
        print("  %-35s [resolution='high']: %s"%(quote(i),background_images[i]['high']))
    except:
        pass
    
print('\nRestart Python for Cartopy to find the new images.\n')


background_images_dir: 
   /home/jovyan/users_conda_envs/stare-1/lib/python3.8/site-packages/cartopy/data/raster/natural_earth
backed up: 
   /home/jovyan/users_conda_envs/stare-1/lib/python3.8/site-packages/cartopy/data/raster/natural_earth/images.json.sav
zipfiles stored to ./tmp/
File ./tmp/HYP_50M_SR_W.zip not found. Fetching.
File ./tmp/NE1_50M_SR_W.zip not found. Fetching.
File ./tmp/HYP_HR_SR_OB_DR.zip not found. Fetching.
File ./tmp/NE1_HR_LC_SR_W_DR.zip not found. Fetching.
NE1_HR_LC_SR_W_DR.tif probably already installed. Skipping copy.

Background images loaded:
  'ne_shaded'                         [resolution='low']:  50-natural-earth-1-downsampled.png
  'hypso_50m_sr_w'                    [resolution='low']:  HYP_50M_SR_W.tif
  'ne1_50m_sr_w'                      [resolution='low']:  NE1_50M_SR_W.tif
  'hypso_10m_sr_w_ob_dr'              [resolution='low']:  HYP_HR_SR_OB_DR.tif
  'ne1_10m_sr_w_dr'                   [resolution='low']:  NE1_HR_LC_SR_W_DR.tif

Restart Pytho